### Libraries

In [2]:
library(tuneR)
library(signal)
library(seewave)
library(wavelets)
library(lintr)


## Identifying bird songs timestamps (in the original propagation tape)

In [3]:
setwd("/home/baptiste/MASTER1/S2/DataMining/project/") 
bande_propag <- readWave("./data/Propagation_experiments/bande_propag.wav")


In [4]:
waveform <- bande_propag@left / (2^(bande_propag@bit - 1))  # Normalize amplitude


In [5]:
lowcut <- 3000  
highcut <- 4500  

Wn <- c(lowcut, highcut) / (bande_propag@samp.rate/2)

bf <- butter(n = 2, Wn, type = "pass")

filtered_wave <- filter(bf, waveform)


In [6]:
threshold <- 0.3  # Amplitude threshold (0.3)
above_threshold <- which(abs(filtered_wave) > threshold)  # Indices where the amplitude exceeds threshold


In [7]:
# Minimum gap between distinct signals (Manual adjustment)
min_gap <- 3
# Convert the indices in seconds
start_times <- above_threshold / bande_propag@samp.rate 
# Apply the minimum gap between each signal
distinct_start_times <- c(start_times[1], start_times[which(diff(start_times) > min_gap) + 1]) 


In [8]:
# Store the results
results <- data.frame(Start_Time = distinct_start_times)
print(results)
write.csv(results, "./data/Propagation_experiments/timestamps.csv", row.names = FALSE)



    Start_Time
1     22.12576
2     28.28084
3     36.48009
4     44.02317
5     51.97204
6     59.97846
7     65.95293
8     72.30483
9     78.89853
10    85.52510
11    92.73494
12    99.55782
13   107.66574
14   115.65129
15   123.44408
16   132.29002
17   138.54794
18   145.97955
19   152.43145
20   159.29850
21   165.92998
22   173.01308
23   181.23687
24   189.05215
25   197.37156
26   205.97556
27   212.16129
28   218.54896
29   224.70209
30   232.06107
31   244.70256
32   251.35955
33   259.64424
34   267.97200
35   276.45635
36   285.93730
37   292.71964
38   299.02744
39   305.25642
40   312.14066
41   319.29023
42   326.19274
43   333.94594
44   341.51193
45   349.98882
46   357.98188
47   364.39900
48   371.04279
49   377.23442
50   384.18302
51   397.61252
52   404.84515
53   413.07571
54   420.35889
55   428.28066
56   436.68048
57   443.66147
58   450.10909
59   456.21982
60   462.94054
61   469.68646
62   477.01839
63   485.66596
64   493.94111
65   501.61707
66   510.2

## Extracting the signals

#### Filter the .wav files

In [4]:
filter_audio <- function(audio){
    lowcut <- 500  
    highcut <- 6000  

    filtered_audio <- ffilter(audio, f = audio@samp.rate, from = lowcut, to = highcut, bandpass = TRUE, wn = "hanning", output = "Wave")
    
    max_val <- max(abs(filtered_audio@left))
    scaling_factor <- 32767 / max_val
    filtered_audio@left <- filtered_audio@left * scaling_factor
    filtered_audio@left <- round(filtered_audio@left)

    return(filtered_audio)
}


In [7]:
audio <- readWave("./data/Propagation_experiments/bande_propag.wav")
audio <- filter_audio(audio)
writeWave(audio, "./data/Propagation_experiments/bande_propag_filtered.wav")


In [5]:
audio <- readWave("./data/Propagation_experiments/enregistrements_propagation_Lipaugus/12_5_m_propag.WAV")
audio <- filter_audio(audio)
writeWave(audio, "./data/Propagation_experiments/enregistrements_propagation_Lipaugus/12_5_m_propag_filtered.WAV")


In [6]:
audio <- readWave("./data/Propagation_experiments/enregistrements_propagation_Lipaugus/25_m_propag.WAV")
audio <- filter_audio(audio)
writeWave(audio, "./data/Propagation_experiments/enregistrements_propagation_Lipaugus/25_m_propag_filtered.WAV")


In [7]:
audio <- readWave("./data/Propagation_experiments/enregistrements_propagation_Lipaugus/50_m_propag.WAV")
audio <- filter_audio(audio)
writeWave(audio, "./data/Propagation_experiments/enregistrements_propagation_Lipaugus/50_m_propag_filtered.WAV")


In [8]:
audio <- readWave("./data/Propagation_experiments/enregistrements_propagation_Lipaugus/100_m_propag.WAV")
audio <- filter_audio(audio)
writeWave(audio, "./data/Propagation_experiments/enregistrements_propagation_Lipaugus/100_m_propag_filtered.WAV")


In [5]:
audio <- readWave("./data/Propagation_experiments/enregistrements_propagation_Lipaugus/200_m_propag.WAV")
audio <- filter_audio(audio)
writeWave(audio, "./data/Propagation_experiments/enregistrements_propagation_Lipaugus/200_m_propag_filtered.WAV")



#### Cutting segments function

In [12]:
time_minus <- 0.1
time_plus <- 1.3


In [13]:
cut_segments <- function(timestamps, audio, distance, output_dir, noise_output_dir) {
    individuals <- c("O", "K", "L", "M", "N", "NMA", "NMC", "NMD", "NME", "Z")
    for (i in 1:nrow(timestamps)) {
        actual_timestamp <- timestamps[[i, 1]]
        start_time <- actual_timestamp - time_minus
        end_time <- actual_timestamp + time_plus
        
        noise_start_time <- end_time + 0.6
        noise_end_time <- noise_start_time + (time_minus + time_plus)
        name_individual <- individuals[((i - 1) %/% 10) + 1]

        segment <- extractWave(audio, from = start_time, to = end_time, xunit="time")
        noise_segment <- extractWave(audio, from = noise_start_time, to = noise_end_time, xunit="time")

        output_name <- paste0(output_dir, name_individual, "_segment_", i, "_", distance, "m.wav")
        noise_output_name <- paste0(noise_output_dir, name_individual, "_noise_", i, "_", distance, "m.wav")
        writeWave(segment, output_name)
        writeWave(noise_segment, noise_output_name)
    }
}


### From the original propagation tape

In [6]:
audio <- readWave("./data/Propagation_experiments/bande_propag_filtered.wav")
audio_duration <- length(audio@left) / audio@samp.rate 
print(paste("Audio duration:", audio_duration, "seconds"))

timestamps <- read.csv("./data/Propagation_experiments/timestamps.csv")
output_dir <- "./data/Propagation_experiments/individuals_bande_propag/"

if (!dir.exists(output_dir)) {
  dir.create(output_dir, recursive = FALSE)
}

individuals <- c("O", "K", "L", "M", "N", "NMA", "NMC", "NMD", "NME", "Z")

for (i in 1:nrow(timestamps)) {
  actual_timestamp <- timestamps[[i, 1]]
  start_time <- actual_timestamp - time_minus
  end_time <- actual_timestamp + time_plus

  name_individual <- individuals[((i - 1) %/% 10) + 1]

  segment <- extractWave(audio, from = start_time, to = end_time, xunit="time")

  output_name <- paste0(output_dir, name_individual, "_segment_", i, ".wav")
  writeWave(segment, output_name)
}


[1] "Audio duration: 776.782947845805 seconds"


### 12.5 meters
/\ = 25.08s.  
/\ ---> beginning of original tape = 112s.  
Time to be added = 137.08s.

In [7]:
audio <- readWave("./data/Propagation_experiments/enregistrements_propagation_Lipaugus/12_5_m_propag_filtered.WAV")
audio_duration <- length(audio@left) / audio@samp.rate 
print(paste("Audio duration:", audio_duration, "seconds"))

timestamps <- read.csv("./data/Propagation_experiments/timestamps.csv")
output_dir <- "./data/Propagation_experiments/individuals_propagation/12_5_m/"
noise_output_dir <- "./data/Propagation_experiments/noise_propagation/noise_12_5_m/"

if (!dir.exists(output_dir)) {
  dir.create(output_dir, recursive = FALSE)
}

if (!dir.exists(noise_output_dir)) {
  dir.create(noise_output_dir, recursive = FALSE)
}

timestamps <- timestamps + 137.08

cut_segments(timestamps, audio, "12-5", output_dir, noise_output_dir)


[1] "Audio duration: 981.286893424036 seconds"


### 25 meters
/\ = 26.92s.  
Time to be added = 138.92 

In [8]:
audio <- readWave("./data/Propagation_experiments/enregistrements_propagation_Lipaugus/25_m_propag_filtered.WAV")
audio_duration <- length(audio@left) / audio@samp.rate 
print(paste("Audio duration:", audio_duration, "seconds"))

timestamps <- read.csv("./data/Propagation_experiments/timestamps.csv")
output_dir <- "./data/Propagation_experiments/individuals_propagation/25_m/"
noise_output_dir <- "./data/Propagation_experiments/noise_propagation/noise_25_m/"

if (!dir.exists(output_dir)) {
  dir.create(output_dir, recursive = FALSE)
}

if (!dir.exists(noise_output_dir)) {
  dir.create(noise_output_dir, recursive = FALSE)
}

timestamps <- timestamps + 138.92

cut_segments(timestamps, audio, "25", output_dir, noise_output_dir)


[1] "Audio duration: 980.062040816327 seconds"


### 50 meters
/\ = 24.735  
Time to be added = 136.735

In [9]:
audio <- readWave("./data/Propagation_experiments/enregistrements_propagation_Lipaugus/50_m_propag_filtered.WAV")
audio_duration <- length(audio@left) / audio@samp.rate 
print(paste("Audio duration:", audio_duration, "seconds"))

timestamps <- read.csv("./data/Propagation_experiments/timestamps.csv")
output_dir <- "./data/Propagation_experiments/individuals_propagation/50_m/"
noise_output_dir <- "./data/Propagation_experiments/noise_propagation/noise_50_m/"

if (!dir.exists(output_dir)) {
  dir.create(output_dir, recursive = FALSE)
}

if (!dir.exists(noise_output_dir)) {
  dir.create(noise_output_dir, recursive = FALSE)
}


timestamps <- timestamps + 136.735

cut_segments(timestamps, audio, "50", output_dir, noise_output_dir)


[1] "Audio duration: 976.428117913832 seconds"


### 100 meters
/\ = 42.8s.  
Time to be added = 154.8s.

In [10]:
audio <- readWave("./data/Propagation_experiments/enregistrements_propagation_Lipaugus/100_m_propag_filtered.WAV")
audio_duration <- length(audio@left) / audio@samp.rate 
print(paste("Audio duration:", audio_duration, "seconds"))

timestamps <- read.csv("./data/Propagation_experiments/timestamps.csv")
output_dir <- "./data/Propagation_experiments/individuals_propagation/100_m/"
noise_output_dir <- "./data/Propagation_experiments/noise_propagation/noise_100_m/"

if (!dir.exists(output_dir)) {
  dir.create(output_dir, recursive = FALSE)
}

if (!dir.exists(noise_output_dir)) {
  dir.create(noise_output_dir, recursive = FALSE)
}

timestamps <- timestamps + 154.8

cut_segments(timestamps, audio, "100", output_dir, noise_output_dir)


[1] "Audio duration: 999.886077097506 seconds"


### 200 meters
/\ = 66.47s.  
Time to be added = 178.47

In [11]:
audio <- readWave("./data/Propagation_experiments/enregistrements_propagation_Lipaugus/200_m_propag_filtered.WAV")
audio_duration <- length(audio@left) / audio@samp.rate
print(paste("Audio duration:", audio_duration, "seconds"))

timestamps <- read.csv("./data/Propagation_experiments/timestamps.csv")
output_dir <- "./data/Propagation_experiments/individuals_propagation/200_m/"
noise_output_dir <- "./data/Propagation_experiments/noise_propagation/noise_200_m/"

if (!dir.exists(output_dir)) {
  dir.create(output_dir, recursive = FALSE)
}

if (!dir.exists(noise_output_dir)) {
  dir.create(noise_output_dir, recursive = FALSE)
}

timestamps <- timestamps + 178.47

cut_segments(timestamps, audio, "200", output_dir, noise_output_dir)


[1] "Audio duration: 1053.28036281179 seconds"
